In [1]:
import pandas as pd
from gensim.models import Word2Vec
import fasttext
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.metrics import f1_score, accuracy_score

In [2]:
#load data
data_hwbst = {
    'min1000': pd.read_csv('csv_files/min1000.hwbst.csv', sep=';'),
    'min200': pd.read_csv('csv_files/min200.hwbst.csv', sep=';'),
    'min30': pd.read_csv('csv_files/min30.hwbst.csv', sep=';'),
    'min8': pd.read_csv('csv_files/min8.hwbst.csv', sep=';', error_bad_lines=False)
}

data_wbst = {
    'min1000': pd.read_csv('csv_files/min1000.wbst.csv', sep=';'),
    'min200': pd.read_csv('csv_files/min200.wbst.csv', sep=';'),
    'min30': pd.read_csv('csv_files/min30.wbst.csv', sep=';'),
    'min8': pd.read_csv('csv_files/min8.wbst.csv', sep=';')
}

b'Skipping line 711: expected 6 fields, saw 7\nSkipping line 36520: expected 6 fields, saw 7\nSkipping line 42179: expected 6 fields, saw 7\nSkipping line 83836: expected 6 fields, saw 7\nSkipping line 97058: expected 6 fields, saw 7\n'


In [3]:
#load models fastext
models_fasttext = {
    'model_train_base_bow': fasttext.load_model("models/model_train_base_bow.bin"),
    'model_train_base_skip': fasttext.load_model("models/model_train_base_skip.bin"),
    'model_train_clean_bow': fasttext.load_model("models/model_train_clean_bow.bin"),
    'model_train_clean_skip': fasttext.load_model("models/model_train_clean_skip.bin"),
    'model_wiki_base_bow': fasttext.load_model("models/model_wiki_base_bow.bin"),
    'model_wiki_base_skip': fasttext.load_model("models/model_wiki_base_skip.bin"),
    'model_wiki_clean_bow': fasttext.load_model("models/model_wiki_clean_bow.bin"),
    'model_wiki_clean_skip': fasttext.load_model("models/model_wiki_clean_skip.bin")
}

In [4]:
models_worldvec = {
    'word2vec_train_base': Word2Vec.load("models/word2vec_train_base.model"),
    'word2vec_train_clean': Word2Vec.load("models/word2vec_train_clean.model"),
    'word2vec_wiki_base': Word2Vec.load("models/word2vec_wiki_base.model"),
    'word2vec_wiki_clean': Word2Vec.load("models/word2vec_wiki_clean.model")
}

In [5]:
import warnings
warnings.filterwarnings('ignore')

In [12]:
print("HWBST test")
for dataset, df in data_hwbst.items():
    print(f'Testing {dataset}')
    for name, model in models_worldvec.items():
        prediction = []
        for _, row in df.iterrows():
            question = row['pytanie']
            odp = row['odpowiedź']
            term = 0
            similarities = {}
            try:
                cosine_similarity(model.wv[question].reshape(1, -1), model.wv[odp].reshape(1, -1))[0][0]
            except KeyError:
                continue
            try:
                similarities[row['termin 1']] = cosine_similarity(model.wv[question].reshape(1, -1), model.wv[row['termin 1']].reshape(1, -1))[0][0]
            except KeyError:
                similarities[row['termin 1']] = -2.0
                term += 1
            try:
                similarities[row['termin 2']] = cosine_similarity(model.wv[question].reshape(1, -1), model.wv[row['termin 2']].reshape(1, -1))[0][0]
            except KeyError:
                similarities[row['termin 2']] = -2.0
                term += 1
            try:
                similarities[row['termin 3']] = cosine_similarity(model.wv[question].reshape(1, -1), model.wv[row['termin 3']].reshape(1, -1))[0][0]
            except KeyError:
                similarities[row['termin 3']] = -2.0
                term += 1
            try:
                similarities[row['termin 4']] = cosine_similarity(model.wv[question].reshape(1, -1), model.wv[row['termin 4']].reshape(1, -1))[0][0]
            except KeyError:
                similarities[row['termin 4']] = -2.0
                term += 1
            
            if term != 3:
                best = max(similarities, key=similarities.get)
                if best == odp:
                    prediction.append(1)
                else:
                    prediction.append(0)
        
        y_true = [1 for _ in range(len(prediction))]
        print(f"Testing model {name} F1 score: {f1_score(y_true, prediction)} Accuracy score {accuracy_score(y_true, prediction)}")
        
    for name, model in models_fasttext.items():
        prediction = []
        for _, row in df.iterrows():
            question = row['pytanie']
            odp = row['odpowiedź']
            similarities = {
                row['termin 1']: cosine_similarity(model[question].reshape(1, -1), model[row['termin 1']].reshape(1, -1))[0][0],
                row['termin 2']: cosine_similarity(model[question].reshape(1, -1), model[row['termin 2']].reshape(1, -1))[0][0],
                row['termin 3']: cosine_similarity(model[question].reshape(1, -1), model[row['termin 3']].reshape(1, -1))[0][0],
                row['termin 4']: cosine_similarity(model[question].reshape(1, -1), model[row['termin 4']].reshape(1, -1))[0][0]
            }
            best = max(similarities, key=similarities.get)
            if best == odp:
                prediction.append(1)
            else:
                prediction.append(0)
        
        y_true = [1 for _ in range(len(prediction))]
        print(f"Testing model {name} F1 score: {f1_score(y_true, prediction)} Accuracy score {accuracy_score(y_true, prediction)}")

HWBST test
Testing min1000
Testing model word2vec_train_base F1 score: 0.6849397590361446 Accuracy score 0.5208428767750801
Testing model word2vec_train_clean F1 score: 0.6876800354531354 Accuracy score 0.524018573237653
Testing model word2vec_wiki_base F1 score: 0.6796324208683814 Accuracy score 0.5147297098247164
Testing model word2vec_wiki_clean F1 score: 0.8162504902601647 Accuracy score 0.6895466342702523
Testing model model_train_base_bow F1 score: 0.6092824113718103 Accuracy score 0.4381064972168859
Testing model model_train_base_skip F1 score: 0.649256462290828 Accuracy score 0.48066597704546576
Testing model model_train_clean_bow F1 score: 0.6432533582837666 Accuracy score 0.47411457563666815
Testing model model_train_clean_skip F1 score: 0.6778038296209457 Accuracy score 0.5126348455741097
Testing model model_wiki_base_bow F1 score: 0.752435089875557 Accuracy score 0.6031229988670509
Testing model model_wiki_base_skip F1 score: 0.8496061653538846 Accuracy score 0.738535047534

In [13]:
print("WBST test")
for dataset, df in data_wbst.items():
    print(f'Testing {dataset}')
    for name, model in models_worldvec.items():
        prediction = []
        for _, row in df.iterrows():
            question = row['pytanie']
            odp = row['odpowiedź']
            term = 0
            similarities = {}
            try:
                cosine_similarity(model.wv[question].reshape(1, -1), model.wv[odp].reshape(1, -1))[0][0]
            except KeyError:
                continue
            try:
                similarities[row['termin 1']] = cosine_similarity(model.wv[question].reshape(1, -1), model.wv[row['termin 1']].reshape(1, -1))[0][0]
            except KeyError:
                similarities[row['termin 1']] = -2.0
                term += 1
            try:
                similarities[row['termin 2']] = cosine_similarity(model.wv[question].reshape(1, -1), model.wv[row['termin 2']].reshape(1, -1))[0][0]
            except KeyError:
                similarities[row['termin 2']] = -2.0
                term += 1
            try:
                similarities[row['termin 3']] = cosine_similarity(model.wv[question].reshape(1, -1), model.wv[row['termin 3']].reshape(1, -1))[0][0]
            except KeyError:
                similarities[row['termin 3']] = -2.0
                term += 1
            try:
                similarities[row['termin 4']] = cosine_similarity(model.wv[question].reshape(1, -1), model.wv[row['termin 4']].reshape(1, -1))[0][0]
            except KeyError:
                similarities[row['termin 4']] = -2.0
                term += 1
            
            if term != 3:
                best = max(similarities, key=similarities.get)
                if best == odp:
                    prediction.append(1)
                else:
                    prediction.append(0)
        
        y_true = [1 for _ in range(len(prediction))]
        print(f"Testing model {name} F1 score: {f1_score(y_true, prediction)} Accuracy score {accuracy_score(y_true, prediction)}")
        
    for name, model in models_fasttext.items():
        prediction = []
        for _, row in df.iterrows():
            question = row['pytanie']
            odp = row['odpowiedź']
            similarities = {
                row['termin 1']: cosine_similarity(model[question].reshape(1, -1), model[row['termin 1']].reshape(1, -1))[0][0],
                row['termin 2']: cosine_similarity(model[question].reshape(1, -1), model[row['termin 2']].reshape(1, -1))[0][0],
                row['termin 3']: cosine_similarity(model[question].reshape(1, -1), model[row['termin 3']].reshape(1, -1))[0][0],
                row['termin 4']: cosine_similarity(model[question].reshape(1, -1), model[row['termin 4']].reshape(1, -1))[0][0]
            }
            best = max(similarities, key=similarities.get)
            if best == odp:
                prediction.append(1)
            else:
                prediction.append(0)
        
        y_true = [1 for _ in range(len(prediction))]
        print(f"Testing model {name} F1 score: {f1_score(y_true, prediction)} Accuracy score {accuracy_score(y_true, prediction)}")

WBST test
Testing min1000
Testing model word2vec_train_base F1 score: 0.7016504443504019 Accuracy score 0.5404172099087353
Testing model word2vec_train_clean F1 score: 0.6970190964136005 Accuracy score 0.5349419124218052
Testing model word2vec_wiki_base F1 score: 0.7135444244413851 Accuracy score 0.5546592031610141
Testing model word2vec_wiki_clean F1 score: 0.8087576225112041 Accuracy score 0.678919452325151
Testing model model_train_base_bow F1 score: 0.6530974761305604 Accuracy score 0.4848884492801407
Testing model model_train_base_skip F1 score: 0.668471500695105 Accuracy score 0.5020331904604901
Testing model model_train_clean_bow F1 score: 0.6914503487452363 Accuracy score 0.5284097153533356
Testing model model_train_clean_skip F1 score: 0.6907913669064747 Accuracy score 0.5276404000439608
Testing model model_wiki_base_bow F1 score: 0.7669060848353435 Accuracy score 0.621936476535883
Testing model model_wiki_base_skip F1 score: 0.8527297944773674 Accuracy score 0.743268491042971